In [1]:
import sys
sys.path.append('..')

In [2]:
from training.generators.FileImageGenerator import create_image_lists, get_generators

Using TensorFlow backend.


In [3]:
images_dir = '..\\data\\atari_v1\\screens\\mspacman'
validation_percentage = 25
valid_format = 'png'

In [4]:
imgs_list = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format
)

ValueError: Image directory ..\data\atari_v1\screens\mspacman not found.

In [ ]:
image_size=(160, 210, 3)
batch_size = 5
EPIS_LEN = 6
EPIS_SHIFT = 2

In [ ]:
training_generator, testing_generator = get_generators(
    images_list=imgs_list, 
    image_dir=images_dir, 
    image_size=image_size, 
    batch_size=batch_size, 
    class_mode='episode', 
    episode_len=EPIS_LEN, 
    episode_shift=EPIS_SHIFT
)

In [ ]:
import tensorflow as tf

In [ ]:
train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size, EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size, EPIS_LEN, ) + image_size)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator,     
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size, EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size, EPIS_LEN, ) + image_size)
                  )
)


In [ ]:
dataset_name = 'atari_v1'

In [ ]:
intermediate_dim = 50
input_shape= (EPIS_LEN, )+image_size # image shape
latents_dim = 5

In [ ]:
input_shape

In [ ]:
tdConv = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(**kwds))

In [ ]:
tdDeConv = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2DTranspose(**kwds))

In [ ]:
tdDense = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(**kwds))

In [ ]:
import numpy as np

In [ ]:
c = 3

In [ ]:
enc_mean_lays = [
    tdConv(filters=c, kernel_size=3, strides=(2, 2), activation='relu'),
    tdConv(filters=2*c, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latents_dim)
]

enc_var_lays = [
    tdConv(filters=c, kernel_size=3, strides=(2, 2), activation='relu'),
    tdConv(filters=2*c, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latents_dim)
]



dec_lays = [
    tf.keras.layers.Dense(units=c**3, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(1, c, c, c)),
    tdDeConv(filters=2*c, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tdDeConv(filters=c, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tdDeConv(filters=1, kernel_size=3, strides=(1, 1), padding="SAME"),    
]

In [ ]:
model_name = 'Atari_VAE_tconv'

In [ ]:
experiments_dir='..\\experiments\\'+model_name

In [ ]:
import os
os.path.join(experiments_dir, model_name)

In [ ]:
from training.autoencoders_conditional.Cond_VAE import CondVAE as AE

In [ ]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'input_shape':input_shape,
        'output_shape':latents_dim,
        'layers': enc_mean_lays
    }
    ,
    
     {
        'name': 'inference_logvariance', 
        'input_shape':input_shape,
        'output_shape':latents_dim,
        'layers':enc_var_lays
    }
    ,
    
        {
        'name': 'decoder', 
        'input_shape':latents_dim,
        'output_shape':input_shape,
        'layers':dec_lays
    }
]

In [ ]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [ ]:
_restore

In [ ]:
ae = AE( 
    model_name=model_name, 
    input_shape=input_shape, 
    latents_dim=latents_dim, 
    variables_params=variables_params, 
    restore=None
    )

In [ ]:
import numpy as np
n_generated = 36
#random_latents = np.random.normal(size=[n_generated, latents_dim])
#np.save(file='random_latents.npy', arr=random_latents)
#random_latents = tf.constant(np.load(file='..\\data\\random_latents.npy'))

In [ ]:
random_latents = tf.constant(np.random.normal(size=[n_generated, latents_dim]))
random_latents.shape

In [ ]:
ae.fit(
    train_dataset=train_ds, 
    test_dataset=test_ds,
    instance_name='episode',
    instance_scale=1,
    epochs=10,
    learning_rate=1e-3,
    random_latents=random_latents,
    latents_dim=latents_dim ,
    experiments_dir=experiments_dir,
    gray_plot=True,
    generate_epoch=5,
    save_epoch=5
)